In [1]:
from numpy.random import seed
seed(1)

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [3]:
df = pd.read_csv("exoplanet_data.csv")
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


# Select your features (columns)

In [4]:
# Set features. This will also be used as your x values.
selected_features = df.drop(columns=["koi_disposition"])
# Drop the null columns where all values are null
selected_features = selected_features.dropna(axis='columns', how='all')
# Drop the null rows
selected_features = selected_features.dropna()

# Create a Train Test Split

Use `koi_disposition` for the y values

In [5]:
from sklearn.model_selection import train_test_split

X = pd.get_dummies(selected_features)
y = df["koi_disposition"].values.reshape(-1,1)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [6]:
from sklearn.preprocessing import StandardScaler
X_scaler = StandardScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [7]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

c:\Users\david\Anaconda3\envs\PythonAdv2\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [8]:
from tensorflow.keras.utils import to_categorical
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

# Train the Model


In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [10]:
# Create model and add layers
model = Sequential()
model.add(Dense(units=300, activation='relu', input_dim=X_train_scaled.shape[1]))
model.add(Dense(units=300, activation='relu'))
model.add(Dense(units=y_train_categorical.shape[1], activation='softmax'))

In [11]:
X_train_scaled.shape

(5243, 40)

In [12]:
y_train_categorical.shape

(5243, 3)

In [13]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 300)               12300     
_________________________________________________________________
dense_1 (Dense)              (None, 300)               90300     
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 903       
Total params: 103,503
Trainable params: 103,503
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=250,
    shuffle=True,
    verbose=2
)

Epoch 1/250
164/164 - 2s - loss: 0.3885 - accuracy: 0.8404
Epoch 2/250
164/164 - 0s - loss: 0.2958 - accuracy: 0.8779
Epoch 3/250
164/164 - 0s - loss: 0.2590 - accuracy: 0.8850
Epoch 4/250
164/164 - 0s - loss: 0.2549 - accuracy: 0.8919
Epoch 5/250
164/164 - 0s - loss: 0.2354 - accuracy: 0.8909
Epoch 6/250
164/164 - 0s - loss: 0.2219 - accuracy: 0.9027
Epoch 7/250
164/164 - 0s - loss: 0.2149 - accuracy: 0.9062
Epoch 8/250
164/164 - 0s - loss: 0.2052 - accuracy: 0.9107
Epoch 9/250
164/164 - 0s - loss: 0.2050 - accuracy: 0.9081
Epoch 10/250
164/164 - 0s - loss: 0.2102 - accuracy: 0.9084
Epoch 11/250
164/164 - 0s - loss: 0.1994 - accuracy: 0.9115
Epoch 12/250
164/164 - 0s - loss: 0.1964 - accuracy: 0.9125
Epoch 13/250
164/164 - 0s - loss: 0.1897 - accuracy: 0.9180
Epoch 14/250
164/164 - 0s - loss: 0.1825 - accuracy: 0.9178
Epoch 15/250
164/164 - 0s - loss: 0.1764 - accuracy: 0.9203
Epoch 16/250
164/164 - 0s - loss: 0.1669 - accuracy: 0.9256
Epoch 17/250
164/164 - 0s - loss: 0.1672 - accura

# Performance Matrix

In [16]:
# Save the model
model.save("sequential.h5")

In [17]:
# Load the model
from tensorflow.keras.models import load_model
model = load_model("sequential.h5")

In [18]:
# Evaluate the model using the training data
model_loss, model_accuracy = model.evaluate(X_test_scaled, y_test_categorical, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

55/55 - 1s - loss: 1.2972 - accuracy: 0.8827
Loss: 1.2972373962402344, Accuracy: 0.8827230930328369
